In [18]:
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import numpy as np

In [25]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

train_images_tensor = torch.Tensor(train_images.transpose((0, 3, 1, 2))) # transform to torch tensor
train_labels = torch.from_numpy(train_labels.flatten().astype(np.int_))

dataset = torch.utils.data.TensorDataset(train_images_tensor, train_labels) # create your datset
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32) # create your dataloader

test_images_tensor = torch.Tensor(test_images.transpose((0, 3, 1, 2))) # transform to torch tensor
test_labels = torch.from_numpy(test_labels.flatten().astype(np.int_))

testset = torch.utils.data.TensorDataset(test_images_tensor, test_labels) # create your datset
testloader = torch.utils.data.DataLoader(dataset, batch_size=32) # create your dataloader

In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.fc1 = nn.Linear(1024, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        #x = self.pool(F.relu(self.conv1(x)))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

In [21]:
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             896
         MaxPool2d-2           [-1, 32, 15, 15]               0
            Conv2d-3           [-1, 64, 13, 13]          18,496
         MaxPool2d-4             [-1, 64, 6, 6]               0
            Conv2d-5             [-1, 64, 4, 4]          36,928
            Linear-6                   [-1, 64]          65,600
            Linear-7                   [-1, 10]             650
Total params: 122,570
Trainable params: 122,570
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.38
Params size (MB): 0.47
Estimated Total Size (MB): 0.86
----------------------------------------------------------------


In [22]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [24]:
for epoch in range(10):  # loop over the dataset multiple times
    for data in dataloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

Finished Training


In [26]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
correct/total

0.75638